## 1.Edit distance with matrix
> 参考视频：https://www.youtube.com/watch?v=We3YDTzNXEk

In [3]:
import numpy as np

In [10]:
class EditDistance:
    def __init__(self):
        self.solution = {}
    @staticmethod
    def edit_distance_with_matrix(str1, str2):
        len_1, len_2 = len(str1) + 1, len(str2) + 1
        if str1 and str2:
            matrix = np.zeros((len_2, len_1), dtype=np.int)
            matrix[0, :] = np.arange(len_1)
            matrix[:, 0] = np.arange(len_2)
            for i, char1 in enumerate(str2):
                for j, char2 in enumerate(str1):
                    if char1 == char2:
                        matrix[i + 1, j + 1] = matrix[i, j]
                    else:
                        matrix[i + 1, j + 1] = min(matrix[i, j + 1], matrix[i, j], matrix[i + 1, j]) + 1

            return matrix[len_2 -1 , len_1 - 1], matrix
        else:
            return len_1 -1  if str1 else len_2 - 1
    
    def get_matrix_solution(self, matrix):
        """
        Up refers to inserting; left refers to deleting; digonal refers to substituting.
        """
        r, c = matrix.shape
        if r < 2:
            for _ in matrix[0, :c-1]:
                self.solution[_+1] = f"delete str1.({_})"
            return
        elif c < 2:
            for _ in matrix[:r-1, 0]:
                self.solution[_ + 1] = f"str1.({0}) + str2.({_})"
            return

        row, column = np.array(matrix.shape) - 1
        target = matrix[row, column]
        up = row - 1, column
        left = row, column - 1
        diag = row - 1, column - 1

        temp = min(matrix[diag], matrix[up], matrix[left])

        # 以下if控制语句的顺序不可变，即从up -> left -> diag顺序判断
        if target == temp + 1:
            if target == matrix[up] + 1:
                self.solution[target] = f"str1.({column - 1}) + str2.({row - 1})"
                return self.get_matrix_solution(matrix[:row, :])
            elif target == matrix[left] + 1:
                self.solution[target] = f"delete str1.({column - 1})"
                return self.get_matrix_solution(matrix[:, :column])
            elif target == matrix[diag] + 1:
                self.solution[target] = f"str1.({column - 1}) -> str2.({row - 1})"
                return self.get_matrix_solution(matrix[:row,:column])
        return self.get_matrix_solution(matrix[:row,:column])
    
    def get_solution(self):
        return sorted(self.solution.items())

In [12]:
test = EditDistance()
step, matrix_ = test.edit_distance_with_matrix('garcg', 'kacfg')
test.get_matrix_solution(matrix_)
print(matrix_)
test.get_solution()

[[0 1 2 3 4 5]
 [1 1 2 3 4 5]
 [2 2 1 2 3 4]
 [3 3 2 2 2 3]
 [4 4 3 3 3 3]
 [5 4 4 4 4 3]]


[(1, 'str1.(0) -> str2.(0)'),
 (2, 'delete str1.(2)'),
 (3, 'str1.(3) + str2.(3)')]

In [7]:
test = EditDistance()
step, matrix_ = test.edit_distance_with_matrix('*execution', 'inte*ntion')
test.get_matrix_solution(matrix_)
test.get_solution()

[(1, 'str1.(0) -> str2.(0)'),
 (2, 'str1.(1) -> str2.(1)'),
 (3, 'str1.(2) -> str2.(2)'),
 (4, 'str1.(4) -> str2.(4)'),
 (5, 'str1.(5) -> str2.(5)')]

In [8]:
test = EditDistance()
step, matrix_ = test.edit_distance_with_matrix('inte*ntion', '*execution')
test.get_matrix_solution(matrix_)
test.get_solution()

[(1, 'str1.(0) -> str2.(0)'),
 (2, 'str1.(1) -> str2.(1)'),
 (3, 'str1.(2) -> str2.(2)'),
 (4, 'str1.(4) -> str2.(4)'),
 (5, 'str1.(5) -> str2.(5)')]

In [6]:
test_ = EditDistance()
step, matrix_ = test_.edit_distance_with_matrix('ago', 'got')
test_.get_matrix_solution(matrix_)
print(matrix_)
print(step)
test_.get_solution()

[[0 1 2 3]
 [1 1 1 2]
 [2 2 2 1]
 [3 3 3 2]]
2


[(1, 'delete str1.(0)'), (2, 'str1.(2) + str2.(2)')]

### 增加替换步数选择，1或者2

In [28]:
class EditDistance_update:
    def __init__(self, levenshtein=True):
        self.solution = {}
        self.levenshtein = levenshtein
        
    def edit_distance_with_matrix(self, str1, str2):
        """
        @levenshtein: susbstitute cost 2 steps while True, else cost 1 step.
        """
        len_1, len_2 = len(str1)+1, len(str2)+1
        matrix = np.zeros((len_2, len_1), dtype=np.int)
        if str1 and str2:
            matrix[0, :] = np.arange(len_1)
            matrix[:, 0] = np.arange(len_2)
            for i, char1 in enumerate(str2, start=1):
                for j, char2 in enumerate(str1, start=1):
                        
                    left = matrix[i-1, j] + 1
                    top = matrix[i, j-1] + 1
                    if char1 != char2:
                        diag = (matrix[i-1, j-1] + 2) if self.levenshtein else (matrix[i-1, j-1] + 1)
                    else:
                        diag = matrix[i-1, j-1]
                    
                    matrix[i, j] = min(left, top, diag)

            return matrix[len_2-1 , len_1-1], matrix
        else:
            return (len_1,matrix)  if str1 else (len_2,matrix)
        
    def get_matrix_solution(self, matrix):
        """
        Up refers to inserting; left refers to deleting; digonal refers to substituting.
        """
        r, c = matrix.shape
        if r < 2:
            for _ in matrix[0, :c-1]:
                self.solution[_+1] = f"delete str1.({_})"
            return
        elif c < 2:
            for _ in matrix[:r-1, 0]:
                self.solution[_ + 1] = f"insert before str1.({0}) with str2.({_})"
            return

        row, column = np.array(matrix.shape) - 1
        target = matrix[row, column]
        up = row - 1, column
        left = row, column - 1
        diag = row - 1, column - 1

        temp = min(matrix[diag], matrix[up], matrix[left])
        
        flag = 2 if self.levenshtein else 1

        if target != temp:
            # 删除或者插入优先
            if target == matrix[up] + 1:
                self.solution[target] = f"insert after str1.({column - 1}) with str2.({row - 1})"
                return self.get_matrix_solution(matrix[:row, :])
            elif target == matrix[left] + 1:
                self.solution[target] = f"delete str1.({column - 1})"
                return self.get_matrix_solution(matrix[:, :column])
            elif target == matrix[diag] + flag:
                self.solution[target] = f"substitute str1.({column - 1}) with str2.({row - 1})"
                return self.get_matrix_solution(matrix[:row,:column])
        return self.get_matrix_solution(matrix[:row,:column])
    
    def get_solution(self):
        return sorted(self.solution.items())

In [29]:
test_update = EditDistance_update()
step, matrix_ = test_update.edit_distance_with_matrix('helool123', 'welhel123lo')
test_update.get_matrix_solution(matrix_)
test_update.get_solution()

[(1, 'insert before str1.(0) with str2.(0)'),
 (2, 'insert before str1.(0) with str2.(1)'),
 (3, 'insert before str1.(0) with str2.(2)'),
 (4, 'delete str1.(3)'),
 (5, 'delete str1.(4)'),
 (6, 'delete str1.(5)'),
 (7, 'insert after str1.(8) with str2.(9)'),
 (8, 'insert after str1.(8) with str2.(10)')]

In [31]:
test_update1 = EditDistance_update(False)
step, matrix_ = test_update1.edit_distance_with_matrix('helool123', 'welhel123lo')
test_update1.get_matrix_solution(matrix_)
print(matrix_)
test_update1.get_solution()

[[ 0  1  2  3  4  5  6  7  8  9]
 [ 1  1  2  3  4  5  6  7  8  9]
 [ 2  2  1  2  3  4  5  6  7  8]
 [ 3  3  2  1  2  3  4  5  6  7]
 [ 4  3  3  2  2  3  4  5  6  7]
 [ 5  4  3  3  3  3  4  5  6  7]
 [ 6  5  4  3  4  4  3  4  5  6]
 [ 7  6  5  4  4  5  4  3  4  5]
 [ 8  7  6  5  5  5  5  4  3  4]
 [ 9  8  7  6  6  6  6  5  4  3]
 [10  9  8  7  7  7  6  6  5  4]
 [11 10  9  8  7  7  7  7  6  5]]


[(1, 'substitute str1.(0) with str2.(0)'),
 (2, 'substitute str1.(3) with str2.(3)'),
 (3, 'substitute str1.(4) with str2.(4)'),
 (4, 'insert after str1.(8) with str2.(9)'),
 (5, 'insert after str1.(8) with str2.(10)')]

## 2.Edit distance with recursion

In [2]:
from functools import lru_cache

In [3]:
if 'solution_' in dir(): del solution_ 
solution_ = {}
@lru_cache(maxsize=2**10)
def edit_distance(string1, string2):
    if string1 and string2:
    
        tail_s1 = string1[-1]
        tail_s2 = string2[-1]

        candidates = [
            (edit_distance(string1[:-1], string2) + 1, 'DEL {}'.format(tail_s1)),  # string 1 delete tail
            (edit_distance(string1, string2[:-1]) + 1, 'ADD {}'.format(tail_s2)),  # string 1 add tail of string2
        ]

        if tail_s1 == tail_s2:
            both_forward = (edit_distance(string1[:-1], string2[:-1]) + 0, '')
        else:
            both_forward = (edit_distance(string1[:-1], string2[:-1]) + 1, 'SUB {} => {}'.format(tail_s1, tail_s2))

        candidates.append(both_forward)

        min_distance, operation = min(candidates, key=lambda x: x[0])

        solution_[(string1, string2)] = operation 
        return min_distance
    
    return len(string1) if string1 else len(string2)

def stop_edit(str1, str2): return str1 == str2

def parse_solution(solution_dict,string1, string2):
    parsed_solutions = []
    
    while not stop_edit(string1, string2):
        
        if not string1:
            for char in string2:
                parsed_solutions.append(f"ADD {char}")
        if not string2:
            for char in string1:
                parsed_solutions.append(f"DEL {char}")
                
        operation = solution_dict.get((string1, string2), '')
        if 'SUB' in operation:
            string1, string2 = string1[:-1], string2[:-1]
        elif operation == '':
            string1, string2 = string1[:-1], string2[:-1]
        elif 'DEL' in operation:
            string1, string2 = string1[:-1], string2
        elif 'ADD' in operation:
            string1, string2 = string1, string2[:-1]            
            
        parsed_solutions.append(operation)
    return [action for action in parsed_solutions if action]
            

In [4]:
edit_distance('ago', 'got')

2

In [5]:
solution_

{('a', 'g'): 'SUB a => g',
 ('a', 'go'): 'ADD o',
 ('a', 'got'): 'ADD t',
 ('ag', 'g'): '',
 ('ag', 'go'): 'ADD o',
 ('ag', 'got'): 'ADD t',
 ('ago', 'g'): 'DEL o',
 ('ago', 'go'): '',
 ('ago', 'got'): 'ADD t'}

In [6]:
parse_solution(solution_, 'ago', 'got')

['ADD t', 'DEL a']

## 3.Longest common substring with matrix
> 最长公共子串

In [56]:
def longest_substring_matrix(str1, str2):
    """
    Return all the longest common substring of two strings.
    """
    result = []
    row, column = len(str1), len(str2)
    matrix = np.zeros((row, column), dtype=int)
    for i, char1 in enumerate(str1):
        for j, char2 in enumerate(str2):
            if char1 == char2:
                matrix[i, j] = matrix[i-1, j-1] + 1
    print(matrix)
    print(args)
    for arg in args:
        index_ = arg[0]
        result.append(str1[index_ - max_ + 1 : index_ + 1])
    return max_, set(result)

In [57]:
longest_substring_matrix('hellowel', 'hellfadellolflowedlfalowefsello')

[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 2 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0]
 [0 0 3 1 0 0 0 0 2 1 0 1 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 2 1 0]
 [0 0 1 4 0 0 0 0 1 3 0 1 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 3 0]
 [0 0 0 0 0 0 0 0 0 0 4 0 0 0 2 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 4]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 0 4 0 0 1 0 0 0]
 [0 0 2 1 0 0 0 0 2 1 0 1 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 2 1 0]]
[[ 3  3]
 [ 4 10]
 [ 4 30]
 [ 6 16]
 [ 6 24]]


(4, {'ello', 'hell', 'lowe'})

In [58]:
longest_substring_matrix('fish', 'fhish')

[[1 0 0 0 0]
 [0 0 1 0 0]
 [0 0 0 2 0]
 [0 1 0 0 3]]
[[3 4]]


(3, {'ish'})

## 4.Longest common subsequence with matrix
> 最长公共子序列

In [59]:
def longest_subsequence_matrix(str1, str2):
    """
    Return the longest common subsequence of two strings, return one of them if there are mutiple solution with same 
    length.
    """
    str1, str2 = ' '+ str1, ' '+ str2
    index_ = []
    len_1, len_2 = len(str1), len(str2)
    if str1 and str2:
        matrix = np.zeros((len_1, len_2), dtype=np.int)
        for i, char1 in enumerate(str1):
            if i == 0: continue
            for j, char2 in enumerate(str2):
                if j == 0: continue
                if char1 == char2:
                    matrix[i, j] = matrix[i - 1, j - 1] + 1
                else:
                    matrix[i, j] = max(matrix[i, j - 1], matrix[i - 1, j])
                    
        target = np.max(matrix)
        
        for i in range(1, target + 1):
            # 找出第一个加1的数
            # 行小于列，以行索引取最小，反之，以列索引取最小
            flag = 0 if len_1 < len_2 else 1
            temp = min(np.argwhere(matrix == i), key = lambda x: x[flag])
            index_.append(temp)  
            
        result = [str1[i[0]] for i in index_]
        print(matrix)
        return target, result
    else:
        return len_1 -1  if str1 else len_2 - 1

In [60]:
longest_subsequence_matrix('h1e2ll3o2y34o234u', 'helloyou')

[[0 0 0 0 0 0 0 0 0]
 [0 1 1 1 1 1 1 1 1]
 [0 1 1 1 1 1 1 1 1]
 [0 1 2 2 2 2 2 2 2]
 [0 1 2 2 2 2 2 2 2]
 [0 1 2 3 3 3 3 3 3]
 [0 1 2 3 4 4 4 4 4]
 [0 1 2 3 4 4 4 4 4]
 [0 1 2 3 4 5 5 5 5]
 [0 1 2 3 4 5 5 5 5]
 [0 1 2 3 4 5 6 6 6]
 [0 1 2 3 4 5 6 6 6]
 [0 1 2 3 4 5 6 6 6]
 [0 1 2 3 4 5 6 7 7]
 [0 1 2 3 4 5 6 7 7]
 [0 1 2 3 4 5 6 7 7]
 [0 1 2 3 4 5 6 7 7]
 [0 1 2 3 4 5 6 7 8]]


(8, ['h', 'e', 'l', 'l', 'o', 'y', 'o', 'u'])

In [61]:
longest_subsequence_matrix('ABCDEFG', 'BCDAFG')

[[0 0 0 0 0 0 0]
 [0 0 0 0 1 1 1]
 [0 1 1 1 1 1 1]
 [0 1 2 2 2 2 2]
 [0 1 2 3 3 3 3]
 [0 1 2 3 3 3 3]
 [0 1 2 3 3 4 4]
 [0 1 2 3 3 4 5]]


(5, ['B', 'C', 'D', 'F', 'G'])

## 5.dijsktra算法
![](./graph.png)

In [15]:
inf = float('inf')

In [16]:
graph = {
    'start': {'a': 6, 'b': 2},
    'a': {'fin' : 1},
    'b': {'a': 3, 'fin': 5},
    'fin': {}
}
costs = {
    'a': 6,
    'b': 2,
    'fin': inf
}
parents = {
    'a': 'start',
    'b': 'start',
    'fin': None
}
processed = []

In [19]:
# del r
r = ['start']
def find_lowest_cost_node(costs):
    result = list(filter(lambda x: x[0] not in processed, sorted(costs.items(), key=lambda x: x[1])))
    return result[0][0] if result else None

def get_next_(result, key='start'):
    for k, v in result.items():
        if k == key:
            r.append(v)
            get_next_(result, v)
    return r
        
def dijsktra_algorithm(graph=graph, costs=costs, parents=parents):
    node = find_lowest_cost_node(costs)
    result = {}
    while node is not None:
        cost = costs[node]
        neighbors = graph[node]
        for n in neighbors.keys():
            new_cost = cost + neighbors[n]
            if costs[n] > new_cost:
                costs[n] = new_cost
                parents[n] = node
        processed.append(node)
        node = find_lowest_cost_node(costs)
    result = {v:k for k,v in parents.items()}
    result = get_next_(result)
    return ' -> '.join(result), costs['fin']

In [20]:
dijsktra_algorithm()

('start -> b -> a -> fin', 6)